<a href="https://colab.research.google.com/github/sightes/RNA-UBO-2023/blob/main/ejemplo_sabado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
url='https://raw.githubusercontent.com/sightes/RNA-UBO-2023/main/IBM_2006-01-01_to_2018-01-01.csv'

In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [24]:
dataset=pd.read_csv(url)
dataset.head(5)
training_set=dataset.iloc[:,1:2].values
sc=MinMaxScaler(feature_range=(0,1))
train_set_sc=sc.fit_transform(training_set)
X_train=[]
y_train=[]
for i in range(60,1200):
  X_train.append(train_set_sc[i-60:i,0])
  y_train.append(train_set_sc[i,0])
X_train,y_train=np.asarray(X_train),np.asarray(y_train)
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1]))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_set_sc[:,0])

In [29]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM
regresor=Sequential()
regresor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
regresor.add(Dropout(0.2))
regresor.add(LSTM(units=50))
regresor.add(Dropout(0.2))
regresor.add(Dense(units=1))
regresor.compile(optimizer='adam',loss='mean_squared_error')
regresor.fit(X_train,y_train,epochs=50,batch_size=50)

In [ ]:
X_train[0],y_train[0]

In [ ]:
a=regresor.predict(X_train[1].reshape(1,-1))

In [ ]:
a.tolist()[0][0]

In [53]:
sc.inverse_transform(np.asarray([a.tolist()[0][0]]).reshape(-1,1))

array([[82.94180419]])